In [1]:
import pandas as pd
import numpy as np
import pickle
root="data/"

In [2]:
df = {}
for k in ["train", "val", "test"]:
    df[k] = {}
    for c in ["nlfl", "mf"]:
        df[k][c] = pd.read_excel(root+f"{c}_{k}_sample_v3.xlsx", index_col=0) if c == "nlfl" else pd.read_excel(root+f"{c}_features_{k}_task_C1.xlsx", index_col=0)

In [3]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

In [4]:
cols_mf = [c for c in df["train"]["mf"].columns if "linguistic" not in c]

X_train = df["train"]["mf"][cols_mf]
X_val = df["val"]["mf"][cols_mf]
X_test = df["test"]["mf"][cols_mf]

X_train = X_train.loc[:, ~X_train.columns.duplicated()]
X_val = X_val.loc[:, ~X_val.columns.duplicated()]
X_test = X_test.loc[:, ~X_test.columns.duplicated()]

y_train = (df["train"]["nlfl"].set_index("id").loc[X_train.index]["label"]).apply(int)
y_val = (df["val"]["nlfl"].set_index("id").loc[X_val.index]["label"]).apply(int)
y_test = (df["test"]["nlfl"].set_index("id").loc[X_test.index]["label"]).apply(int)

In [5]:
train_test_sample = pd.concat([
    pd.concat([X_train, y_train], axis=1), 
    pd.concat([X_val, y_val], axis=1)
    ], ignore_index=True)
train_test_sample

,contextual<&>Q[rel_subj]+&A+,contextual<&>Q[quién|cuál|qué],contextual<&>Q[ser&posible],contextual<&>A[binary(si|no)],contextual<&>Q[por&qué&equivocado],contextual<&>Q[ser*&bien],contextual<&>A[porque|binary(si|no)],contextual<&>Q[propn]+&A+,contextual<&>Q[quién|cuál],contextual<&>Q[(ser*&correcto)|(tener&razón)],...,semantic<&>ratio_ud,semantic<&>ratio_slang,semantic<&>ratio_keywords,semantic<&>ratio_faces,traditional<&>ratio_vowel,traditional<&>ratio_no_numbers,traditional<&>ratio_punct,traditional<&>exist_numbs,traditional<&>max_len_number,label
0,0,1,0,0,0,0,0,0,1,0,...,0.000000,0.000000,0.000000,1.000000,0.333333,1.000000,0.000000,1,1,0
1,0,0,0,0,0,0,0,0,1,0,...,0.244898,0.022222,0.022222,0.044444,0.421053,0.555556,0.032000,1,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0.437500,0.066667,0.066667,0.000000,0.428571,0.533333,0.045455,1,3,0
3,4,0,0,0,0,0,0,4,1,0,...,0.333333,0.133333,0.000000,0.000000,0.456522,0.800000,0.037736,1,2,0
4,1,0,0,0,0,0,0,1,1,0,...,0.384615,0.000000,0.000000,0.083333,0.523810,0.500000,0.027027,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14452,3,1,0,1,0,0,0,3,1,0,...,0.046512,0.000000,0.000000,0.050000,0.463235,1.025000,0.000000,1,2,0
14453,2,1,0,1,0,0,0,2,1,0,...,0.058824,0.022222,0.022222,0.044444,0.445122,1.022222,0.000000,1,2,0
14454,3,1,0,1,0,0,0,3,1,0,...,0.036364,0.039216,0.000000,0.019608,0.483146,1.019608,0.000000,1,2,0
14455,0,1,0,0,0,0,0,0,1,0,...,0.000000,0.100000,0.100000,0.100000,0.468085,1.000000,0.000000,0,0,0


In [6]:
F_features = np.load("output/mf_features.npy")

In [7]:
features, counts = np.unique(F_features, return_counts=True)
k = 5
new_best_features = features[counts>=k]

X_train_val = train_test_sample.drop(columns="label")
y_train_val = train_test_sample["label"]

clf = DecisionTreeClassifier(random_state=42, max_depth=5)
clf.fit(X_train_val[new_best_features], y_train_val)
print(clf.score(X_train_val[new_best_features], y_train_val))
print(classification_report(y_test, clf.predict(X_test[new_best_features]), digits=4))
o = classification_report(y_test, clf.predict(X_test[new_best_features]), digits=4, output_dict=True)
o

0.9352562772359411
              precision    recall  f1-score   support

           0     0.9192    0.9667    0.9423       541
           1     0.8333    0.6618    0.7377       136

    accuracy                         0.9055       677
   macro avg     0.8762    0.8142    0.8400       677
weighted avg     0.9019    0.9055    0.9012       677



{'0': {'precision': 0.9191564147627417,
  'recall': 0.966728280961183,
  'f1-score': 0.9423423423423425,
  'support': 541},
 '1': {'precision': 0.8333333333333334,
  'recall': 0.6617647058823529,
  'f1-score': 0.7377049180327869,
  'support': 136},
 'accuracy': 0.9054652880354506,
 'macro avg': {'precision': 0.8762448740480375,
  'recall': 0.8142464934217679,
  'f1-score': 0.8400236301875648,
  'support': 677},
 'weighted avg': {'precision': 0.9019157366617083,
  'recall': 0.9054652880354506,
  'f1-score': 0.9012334949182663,
  'support': 677}}